下記のリンクをクリックするとGoogle Colabで実行することが出来ます  
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/2Nike2/LangChainPractice/blob/main/00_03_use_retriever.ipynb)

### (事前準備: OpenAI APIキーの設定)
OpenAI APIを使う為のAPIキーを設定します  
このAPIキーについては、OpenAIのサイトで取得することが出来ます  
https://platform.openai.com/api-keys  
APIキーについては公開しないように注意してください  

In [ ]:
import os

# ここにあなたのOpenAIのAPIキーを入力してください
openai_api_key = 'yourapikey'

# 環境変数にAPIキーがまだ設定されていないならばAPIキーを設定
if os.getenv('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = openai_api_key


## Web文書をソースとしたリトリーバを使ったRAG
検索した文書を指示、質問と共にコンテキストとしてLLMに渡すことで正確な回答を得る手法RAG(Retrieval Augmented Generation:検索強化生成)の動作を確認します

### ライブラリのインストール
LangChainのライブラリをインストールします  
またWeb文書を取得するためのライブラリであるBeautifulSoup  
及びベクトルデータベースのfaiss(CPU版)もインストールします  


In [ ]:
!pip install langchain==0.1.4
!pip install langchain-openai==0.0.5
!pip install beautifulsoup4==4.12.3
!pip install faiss-cpu==1.7.4


### モデルの初期化
OpenAI APIを使う為のモデルを初期化します

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()


### Web文書の取得
WebBaseLoaderを使ってWeb文書を取得します

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.aozora.gr.jp/cards/000081/files/43754_17659.html") # 青空文庫 「注文の多い料理店」(宮沢賢治)

docs = loader.load()


### 埋め込みモデル
埋め込みモデルを初期化します
これは今まで使ってきた文章に対して文章を返すモデルと違い、文章に対して埋め込みベクトル(数百~数千個の数値のリスト)を返すモデルです  
個の埋め込みベクトルというのは文章の意味を数値に要約したものと考えられ、複数の埋め込みベクトルの距離や角度を計算することによって文章の意味の近さを捉えられます  
これを利用して指示や質問に対して意味が近かったり関連性が高い文章を探し出すことが出来ます

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


### ベクトルデータベースの用意
上記の埋め込みベクトルを格納して高速に検索するためのベクトルデータベースを用意します  
ベクトルデータベースに登録する文章は全体をそのまま入れるのではなく、何らかの単位(字数、章、ページ区切り等)で分割してからベクトル化して保存することになります

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter()
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=800, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


### チェインを作成
質問への回答をしてもらう為のチェインを作成します

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import  ChatPromptTemplate

template = ChatPromptTemplate.from_template("""\
以下の質問について、与えられたcontextを元に回答してください。
<context>
{context}
</context>

質問: {input}
""")

document_chain = create_stuff_documents_chain(llm, template)


### 文脈を直接与えて回答
上記のチェインで質問と文脈を直接与えたときの挙動を確認します

In [ ]:
from langchain_core.documents import Document

print(document_chain.invoke({
    'input': '玄関の札に出ていたレストランの日本語名は？',
    'context': [Document(page_content="""\
ところがどうも困ったことは、どっちへ行けば戻れるのか、いっこうに見当がつかなくなっていました。
　風がどうと吹いてきて、草はざわざわ、木の葉はかさかさ、木はごとんごとんと鳴りました。
「どうも腹が空いた。さっきから横っ腹が痛くてたまらないんだ。」
「ぼくもそうだ。もうあんまりあるきたくないな。」
「あるきたくないよ。ああ困ったなあ、何かたべたいなあ。」
「喰べたいもんだなあ」
　二人の紳士は、ざわざわ鳴るすすきの中で、こんなことを云いました。
　その時ふとうしろを見ますと、立派な一軒の西洋造りの家がありました。
　そして玄関には

RESTAURANT
西洋料理店
WILDCAT HOUSE
山猫軒

という札がでていました。
「君、ちょうどいい。ここはこれでなかなか開けてるんだ。入ろうじゃないか」
「おや、こんなとこにおかしいね。しかしとにかく何か食事ができるんだろう」
「もちろんできるさ。看板にそう書いてあるじゃないか」
「はいろうじゃないか。ぼくはもう何か喰べたくて倒れそうなんだ。」
　二人は玄関に立ちました。玄関は白い瀬戸の煉瓦で組んで、実に立派なもんです。
　そして硝子の開き戸がたって、そこに金文字でこう書いてありました。

「どなたもどうかお入りください。決してご遠慮はありません」

　二人はそこで、ひどくよろこんで言いました。
「こいつはどうだ、やっぱり世の中はうまくできてるねえ、きょう一日なんぎしたけれど、こんどはこんないいこともある。このうちは料理店だけれどもただでご馳走するんだぜ。」
「どうもそうらしい。決してご遠慮はありませんというのはその意味だ。」
　二人は戸を押して、なかへ入りました。そこはすぐ廊下になっていました。その硝子戸の裏側には、金文字でこうなっていました。\
""")]
}))


### 文章を検索して文脈を取得して回答
上記の方法だと直接人手で与えた文章を使っている為、  
今度はRAGで文章を検索して文脈を取得し、正しく回答できることを確認します

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever(search_kwargs={'k': 3})
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
from pprint import pprint

response = retrieval_chain.invoke({'input': '玄関の札に出ていたレストランの日本語名は？'})

pprint(response)
